# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write complex queries
* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [8]:
#Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [9]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [10]:
cur.execute("""select customerNumber, contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';""").fetchall()

[('141', 'Freyre', 'Diego ')]

In [11]:
#Your code here; use a subquery. No join will be necessary.
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber FROM 
                                     orders
                                     WHERE orderDate = '2003-01-31');""").fetchall()

[('141', 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [12]:
#Your code here
cur.execute("""SELECT productName, COUNT(orderNumber) as number_orders, SUM(quantityOrdered) AS total_unit_sold 
            FROM products
            JOIN orderdetails
            USING(productCode)
            GROUP BY 1
            ORDER BY total_unit_sold DESC LIMIT 5;""").fetchall()

[('1992 Ferrari 360 Spider red', 53, 1808),
 ('1937 Lincoln Berline', 28, 1111),
 ('American Airlines: MD-11S', 28, 1085),
 ('1941 Chevrolet Special Deluxe Cabriolet', 28, 1076),
 ('1930 Buick Marquette Phaeton', 28, 1074)]

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [13]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

cur.execute("""SELECT productName, COUNT(DISTINCT customerNumber) AS total_people_ordered 
            FROM products
            JOIN orderdetails
            USING(productCode)
            JOIN orders
            USING(orderNumber)
            JOIN customers
            USING(customerNumber)
            GROUP BY 1
            ORDER BY total_people_ordered DESC
            LIMIT 5;""").fetchall()

[('1992 Ferrari 360 Spider red', 40),
 ('1934 Ford V8 Coupe', 27),
 ('1952 Alpine Renault 1300', 27),
 ('1972 Alfa Romeo GTA', 27),
 ('Boeing X-32A JSF', 27)]

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [22]:
#Your code here
cur.execute("""SELECT DISTINCT employeeNumber, firstName, lastName, officeCode,
            FROM employees
            JOIN offices o
            USING(officeCode)
            JOIN customers
            USING (postalCode)
            JOIN orders
            USING(customerNumber)
            JOIN orderdetails
            USING(orderNumber) 
            WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) < 20;
                                            """).fetchall()

OperationalError: near "FROM": syntax error

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Who's Customers Have an Average Credit Limit of Over 15K

In [13]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!